In [0]:
from datetime import datetime
import requests
import os
from pyspark.sql import SparkSession

In [0]:
# Cria sessão Spark
spark = SparkSession.builder.getOrCreate()

In [0]:
# =====================================
# PARÂMETROS DO JOB (Widgets)
# =====================================
dbutils.widgets.text("data_ref", "", "Data de Referência (AAAAMMDD)")
dbutils.widgets.text("url_base", "", "URL base (opcional)")
dbutils.widgets.text("destino", "", "Destino no Volume")

In [0]:
# Lendo parâmetros
data_ref = dbutils.widgets.get("data_ref")
url_base = dbutils.widgets.get("url_base")
destino_base = dbutils.widgets.get("destino")

In [0]:
# =====================================
# VALIDAÇÃO E DEFAULTS
# =====================================

# 1) Se não informar data, usa hoje
if not data_ref:
    data_ref = datetime.now().strftime("%Y%m%d")

# 2) Monta a URL final para o .nc
if not url_base:
    url_base = "https://dataserver-coids.inpe.br/queimadas/queimadas/riscofogo_meteorologia/observado/risco_fogo"

# remove / no final
url_base = url_base.rstrip("/")

# URL final do arquivo .nc
url_nc = f"{url_base}/INPE_FireRiskModel_2.2_FireRisk_{data_ref}.nc"

# 3) Destino
if not destino_base:
    destino_base = "/Volumes/datamasters/raw/raw_inpe"

# Caminho final do arquivo dentro do volume
destino_final = f"{destino_base}/INPE_FireRiskModel_2.2_FireRisk_{data_ref}.nc"

In [0]:
# =====================================
# LOG
# =====================================
print("=====================================")
print(f"Data de referência : {data_ref}")
print(f"URL de origem      : {url_nc}")
print(f"Destino final      : {destino_final}")
print("=====================================")

In [0]:
# =====================================
# DOWNLOAD DO ARQUIVO
# =====================================
try:
    response = requests.get(url_nc)

    if response.status_code != 200:
        raise Exception(f"Falha ao baixar arquivo ({response.status_code}): {url_nc}")

    # Grava o binário .nc
    with open(destino_final, "wb") as f:
        f.write(response.content)

    print(f"Arquivo salvo com sucesso: {destino_final} ✅")

except Exception as e:
    raise RuntimeError(f"Erro no download do arquivo INPE (.nc): {e}")

In [0]:
# =====================================
#  LEITURA PARA VERIFICAÇÃO
# =====================================
file_size_kb = round(os.path.getsize(destino_final) / 1024, 2)
print(f"Tamanho do arquivo: {file_size_kb} KB")

print("Observação: arquivos .nc (NetCDF) não são CSV — leitura deve ser feita com XArray ou rioxarray.")